In [1]:
%load_ext autoreload
%autoreload 2

## Imports

In [ ]:
import pandas as pd
import numpy as np
import pickle
import joblib
from envs.film_env import FilmRecEnv
import random 
from config import settings



## Loading

In [ ]:
catalog_df = pd.read_pickle("data/catalog_df.pkl")            # load your DataFrame
with open(settings.aud_model_dir/"rf_model.pkl", "rb") as f:    # load your frozen viewership model
    audience_model = pickle.load(f)
q_model = joblib.load("models/q_model.joblib")

In [4]:
env = FilmRecEnv(
    catalog_df     = catalog_df,
    audience_model = audience_model,
    memory_size    = 5,
    rec_list_size  = 5,
)

## Context and Recommendation

In [5]:
context = {
        "day": 'wed',
        "time":   'night',
        "season": 'Spring'
    }

In [6]:
env.state = env.reset(context)

# 2) Compute Q‐values and mask memory
q_vals = q_model.predict(env.state)            # shape (n_films,)
if env.memory:
    q_vals[env.memory] = -np.inf

# top‐K by Q‐value
idx = np.argpartition(q_vals, -env.K)[-env.K:] # get indices from top-K but not sorted, more efficient
slate = idx[np.argsort(-q_vals[idx])] # sort the top-K indices by Q-value
slate_names = [catalog_df.iloc[i]['title'] for i in slate]
catalog_df.iloc[slate]

,title,BrdCstClassKey,adult,duration,genres,original_language,popularity,release_date,revenue,tmdb_id,vote_average,missing_release_date,missing_tmdb,is_movie
150,Le crime lui va si bien,72,False,94.083333,[],unknown,0.0000,1900-01-01,0.0,NaN,0.0,True,True,True
347,Quatre mariages et un coup de foudre,72,False,87.933333,[],unknown,0.0000,1900-01-01,0.0,NaN,0.0,True,True,True
436,Body Cam,71,False,89.333333,"[{'id': 99, 'name': 'Documentaire'}, {'id': 80...",en,0.1546,1900-01-01,0.0,1013147.0,0.0,True,False,True
256,"Brillante, belle et brisée",72,False,89.000000,"[{'id': 10770, 'name': 'Téléfilm'}, {'id': 53,...",en,0.1850,2021-07-11,0.0,849937.0,4.4,True,False,True
748,Le mystère de Noël,71,False,71.950000,[],fr,0.0000,1900-01-01,0.0,477678.0,0.0,True,False,True
